In [1]:
%pip install transformers datasets

   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ----------------------------- ---------- 7.6/10.4 MB 40.0 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 34.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   ------------------- -------------------- 12.3/25.7 MB 60.6 MB/s eta 0:00:01
   -------------------------------- ------- 21.0/25.7 MB 48.6 MB/s eta 0:00:01
   ---------------------------------------  25.4/25.7 MB 45.7 MB/s eta 0:00:01
   ---------------------------------------- 25.7/25.7 MB 38.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 27.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


El archivo .json tendrá el formato instruction, response


In [ ]:
import json
from datasets import Dataset
from transformers import AutoTokenizer,AutoModelForCausalLM,Trainer,TrainingArguments,DataCollatorForLanguageModeling
#El collator DataCollatorForLanguageModeling lo utilizamos para modelos como Qwen, que aprenden de forma autoregresiva


# 1. Cargar dataset desde archivo JSON
# dataset_path = "/content/sample_data/ejemplo.json"
# with open(dataset_path, "r", encoding="utf-8") as f:
#     data = json.load(f)
# dataset = Dataset.from_list(data)
from datasets import load_dataset

dataset = load_dataset("naklecha/minecraft-question-answer-700k")


# 2. Cargar modelo y tokenizer
model_name = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 3. Asegurar que haya token de padding
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<pad>'})
    model.resize_token_embeddings(len(tokenizer))

# 4. Agregar tokens especiales si no están
special_tokens = ['<|user|>', '<|assistant|>']
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})
model.resize_token_embeddings(len(tokenizer))

# 5. Construcción del prompt
def build_prompt(example):
    prompt = f"<|user|>\n{example['instruction']}\n<|assistant|>\n{example['response']}"
    return {"text": prompt}

dataset = dataset.map(build_prompt, remove_columns=["instruction", "response"])

# 6. Tokenización
def tokenize(example):
    out = tokenizer(
        example["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    out["labels"] = out["input_ids"].copy()
    return out

tokenized_dataset = dataset.map(tokenize, batched=True)

# 7. Dividir en train/test
splits = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_data = splits["train"]
eval_data = splits["test"]

# 8. Collator para entrenamiento causal
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Muy importante: no es masked LM, es causal LM
)

# 9. Configurar entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none"  # Desactiva W&B y otros trackers
)

# 10. Inicializar y entrenar
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=tokenizer, #processing_class="tokenizer"
    data_collator=data_collator
)

trainer.train()

# 11. Guardar modelo fine-tuneado
trainer.save_model("./results/Qwen_finetuned")
tokenizer.save_pretrained("./results/Qwen_finetuned")

c:\Users\gabriel.antonio.edre\Documents\main\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/ejemplo.json'